In [2]:
import pandas as pd
import numpy as np
import plotly.express as px
from dash import dcc, html, Dash, Input, Output
import dash_bootstrap_components as dbc

# Datenvorbereitung
df = pd.read_csv(r"C:\Users\Patrizia_Hutter\DatInfoVis\flugdaten_verwendet.csv", sep=";")
df = df.replace("", pd.NA)
df['Datum'] = pd.to_datetime(df['Datum'], format='%d.%m.%Y', errors='coerce')
df['Jahr'] = df['Datum'].dt.year
df['Monat'] = df['Datum'].dt.month
df['Tag'] = df['Datum'].dt.day
df['Luftfracht'] = df['Luftfracht'].fillna(0).astype(int)

# Vorverarbeitung: Daten vorberechnen
df_flugbewegung_pro_jahr = df.groupby('Jahr', dropna=True)['Flugbewegungen'].sum().reset_index()

df_monate = df.groupby(['Jahr', 'Monat'], dropna=True)[['Luftfracht', 'Flugbewegungen', 'Flugpassagiere']].sum().reset_index()
df_monate['Monat'] = pd.to_datetime(df_monate['Monat'], format='%m').dt.month_name()
df_monate['Monat'] = pd.Categorical(df_monate['Monat'], 
    categories=['January', 'February', 'March', 'April', 'May', 'June', 
                'July', 'August', 'September', 'October', 'November', 'December'], ordered=True)

df_passagier_summary = df.groupby('Jahr')[['Flugpassagiere']].sum().reset_index()

# Farbzuordnung
color_map = {
    2019: "#87CEEB",
    2020: "#5F9EA0",
    2021: "#4682B4",
    2022: "#4169E1",
    2023: "#1E90FF",
    2024: "#0000CD"
}

# Neue Farbzuordnung für Flugbewegungen
flights_colors = {
    2019: "#808080",  # Klassisches Grau
    2020: "#C0C0C0",  # Sehr helles Grau
    2023: "#333333",  # Dunkles Grau
    "default": ["#20B2AA", "#48D1CC", "#5F9EA0", "#66CDAA"]  # Grüntöne für andere Jahre
}

# Dashboard-App
app = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

# Header-Layout
header_layout = html.Div(
    children=[
        dbc.Row([
            dbc.Col(html.Img(src="/assets/Logo_Flughafen.jpg", className="img-fluid", style={"width": "100%"}), width=4),
            dbc.Col(
                html.Div(
                    [
                        html.Div("Flughafen Basel-Mulhouse-Freiburg: Tägliche Flugbewegungen, Passagiere & Fracht",
                                 className="text-white fw-bold", style={"font-size": "24px"}),
                        html.Div(
                            [
                                html.Img(
                                    src="/assets/coloured_stats.jpg",
                                    className="me-2 img-fluid",
                                    style={"height": "20px"}
                                ),
                                html.Span("Dashboard", className="fw-bold text-white", style={"font-size": "18px"})
                            ],
                            className="d-flex align-items-center"
                        )
                    ]
                ), 
                width=6
            ),
            dbc.Col(html.Div(
                [
                    html.Img(src="/assets/home.jpg", className="mx-2 img-fluid", style={"height": "24px"}),
                    html.Img(src="/assets/stats.jpg", className="mx-2 img-fluid", style={"height": "24px"}),
                    html.Img(src="/assets/user.jpg", className="mx-2 img-fluid", style={"height": "24px"}),
                    html.Img(src="/assets/logout.jpg", className="mx-2 img-fluid", style={"height": "24px"})
                 ],
                 className="d-flex justify-content-end align-items-center"
               ),
               width=2,
            ),
        ], className="align-items-center mb-3")
    ],
    style={"background-color": "black"}
)

# Layout der Hauptinhalte
main_content = dbc.Row([
    dbc.Col(
        html.Div(
            [
                html.Label("Jahre auswählen:", style={"color": "white"}),
                dcc.Checklist(
                    id='line_checklist',
                    options=[{'label': str(int(year)), 'value': year} for year in sorted(df['Jahr'].dropna().unique())],
                    value=[2019, 2020],
                    inline=True,
                    inputStyle={"margin-right": "5px"},
                    style={"color": "white"}
                ),
                html.Br(),
                html.Label("Flugbewegungen auswählen:", style={"color": "white"}),
                dcc.RangeSlider(
                    id='scatter_slider',
                    min=0,
                    max=500,
                    step=10,
                    marks={i: str(i) for i in range(0, 501, 50)},
                    value=[0, 500],
                ),
                html.Br(),
                html.Label("Wählen Sie, was im Liniendiagramm angezeigt werden soll:", style={"color": "white"}),
                dcc.Dropdown(
                    id='linechart_dropdown',
                    options=[
                        {'label': 'Nur Luftfracht', 'value': "Luftfracht"},
                        {'label': 'Nur Flugbewegungen', 'value': "Flugbewegungen"},
                        {'label': 'Beides', 'value': "Beides"}
                    ],
                    value="Beides",
                )
            ],
            style={"border": "2px solid rgb(0, 151, 136)", "padding": "30px", "margin-bottom": "20px",
                   "background-color": "black", "margin-left": "-16px", "width": "100%"}
        ),
        width=3,
    ),
    dbc.Col([
        dbc.Row([
            dbc.Col(
                html.Div(
                    dcc.Graph(id="diagramm_bar"),
                    style={"border": "2px solid rgb(0, 151, 136)", "padding": "30px", "background-color": "black"}
                ),
                width=12
            )
        ]),
        dbc.Row([
            dbc.Col(
                html.Div(
                    dcc.Graph(id="bar_chart_passagiers"),
                    style={"border": "2px solid rgb(0, 151, 136)", "padding": "30px", "background-color": "black"}
                ),
                width=12
            )
        ])
    ], width=3),
    dbc.Col([
        dbc.Row([
            dbc.Col(
                html.Div(
                    dcc.Graph(id="polar_chart"),
                    style={"border": "2px solid rgb(0, 151, 136)", "padding": "30px", "background-color": "black"} 
                ),
                width=12
            )
        ]),
        dbc.Row([
            dbc.Col(
                html.Div(
                    dcc.Graph(id="pie_chart"),
                    style={"border": "2px solid rgb(0, 151, 136)", "padding": "30px"}
                ),
                width=12
            )
        ])
    ], width=3),
    dbc.Col([
        dbc.Row([
            dbc.Col(
                html.Div(
                    dcc.Graph(id="diagramm_line"),
                    style={"border": "2px solid rgb(0, 151, 136)", "padding": "30px", "background-color": "black"}
                ),
                width=12
            )
        ]),
        dbc.Row([
            dbc.Col(
                html.Div(
                    dcc.Graph(id="scatter_passagierverkehr"),
                    style={"border": "2px solid rgb(0, 151, 136)", "padding": "30px", "background-color": "black"}
                ),
                width=12
            )
        ])
    ], width=3)
], className="m-4")

# Layout der App
app.layout = html.Div([
    header_layout,
    main_content
], style={"background-color": "black"})

# Callbacks für alle Diagramme
@app.callback(
    Output('diagramm_bar', 'figure'),
    Input('line_checklist', 'value')
)
def update_bar_chart(selected_years):
    filtered_df_bar = df_flugbewegung_pro_jahr[df_flugbewegung_pro_jahr['Jahr'].isin(selected_years)].copy()
    filtered_df_bar['Jahr'] = filtered_df_bar['Jahr'].astype(str)
    fig = px.bar(
        filtered_df_bar,
        x='Jahr',
        y='Flugbewegungen',
        title='Flugbewegungen pro Jahr',
        color='Jahr',
        color_discrete_map={str(k): v for k, v in color_map.items()},
        template="plotly_dark"
    )
    return fig
# Callback für das neue Liniendiagramm
@app.callback(
    Output('diagramm_line', 'figure'),
    Input('linechart_dropdown', 'value'),
    Input('line_checklist', 'value')
)
def update_line_chart(linechart_selection, selected_years):
    df_filtered = df_monate[df_monate['Jahr'].isin(selected_years)].copy()

    if linechart_selection == "Luftfracht":
        value_vars = ['Luftfracht']
    elif linechart_selection == "Flugbewegungen":
        value_vars = ['Flugbewegungen']
    else:
        value_vars = ['Luftfracht', 'Flugbewegungen']

    df_long = df_filtered.melt(
        id_vars=['Jahr', 'Monat'],
        value_vars=value_vars,
        var_name='Kategorie',
        value_name='Wert'
    )

    df_long['Jahr_Kategorie'] = df_long['Jahr'].astype(str) + " - " + df_long['Kategorie']

    flights_index = 0
    final_color_map = {}
    for year in selected_years:
        if year in color_map:
            final_color_map[f"{year} - Luftfracht"] = color_map[year]
            if year in flights_colors:
                final_color_map[f"{year} - Flugbewegungen"] = flights_colors[year]
            else:
                final_color_map[f"{year} - Flugbewegungen"] = flights_colors["default"][flights_index % len(flights_colors["default"])]
                flights_index += 1

    fig_line = px.line(
        df_long,
        x='Monat',
        y='Wert',
        color='Jahr_Kategorie',
        line_group='Jahr_Kategorie',
        markers=True,
        title='Luftfracht und Flugbewegungen',
        color_discrete_map=final_color_map,
        template="plotly_dark"
    )

    fig_line.for_each_trace(lambda t: t.update(
        name=t.name.replace("Luftfracht", "Lf").replace("Flugbewegungen", "Fbw")
    ))

    return fig_line

# Callback für das Kuchendiagramm
@app.callback(
    Output('pie_chart', 'figure'),
    Input('line_checklist', 'value')
)
def update_pie_chart(selected_years):
    filtered_df = df[df['Jahr'].isin(selected_years)].copy()
    summer_passengers_per_year = {}
    rest_passengers_per_year = {}
    for year in selected_years:
        year_df = filtered_df[filtered_df['Jahr'] == year]
        summer_months = year_df[year_df['Monat'].isin([7, 8])]
        rest_of_year = year_df[~year_df['Monat'].isin([7, 8])]
        summer_passengers_per_year[year] = summer_months['Flugpassagiere'].sum()
        rest_passengers_per_year[year] = rest_of_year['Flugpassagiere'].sum()

    total_summer = sum(summer_passengers_per_year.values())
    total_rest = sum(rest_passengers_per_year.values())

    legend_text = ""
    for year in selected_years:
        legend_text += (
            f"<b>{year}:</b><br>"
            f"Sommerferien = {summer_passengers_per_year[year]}<br>"
            f"Restliches Jahr = {rest_passengers_per_year[year]}<br><br>"
        )

    fig = px.pie(
        values=[total_summer, total_rest],
        names=['Sommerferien', 'Restliches Jahr'],
        title="Passagierzahlen Juli/August vs. Rest",
        color_discrete_sequence=["#87CEEB", "#1E90FF"],
        template="plotly_dark"
    )
    fig.update_traces(textinfo='label+value')

    # Layout mit fester Größe
    fig.update_layout(
        annotations=[
            dict(
                text=legend_text,
                x=1, y=0.3,
                xanchor="left",
                yanchor="middle",
                align="left",
                showarrow=False,
                font=dict(size=11, color="white")
            )
        ],
        margin=dict(l=40, r=150, t=50, b=50),  # Linker Abstand für Verschiebung
        title={"text": "Passagierzahlen Juli/August vs. Rest","y":0.9},  # Verschiebt den Titel nach links
        colorway=["#87CEEB", "#1E90FF"],  # Farben für das Diagramm
        width=375,  # Breite des Diagramms in Pixeln
        height=450,  # Höhe des Diagramms in Pixeln
        legend=dict(
        y=1.05,  # Erhöht die Position der Legende (nach oben verschieben)
        yanchor="top",  # Der Anker der Legende wird an der Oberkante ausgerichtet
        x=1,
        xanchor="left"
    )
    )
    return fig

# Callback für Passagierzahlen-Barchart
@app.callback(
    Output('bar_chart_passagiers', 'figure'),
    Input('line_checklist', 'value')
)
def update_bar_chart_passagiers(selected_years):
    df_filtered = df_passagier_summary[df_passagier_summary['Jahr'].isin(selected_years)].copy()
    df_filtered['Jahr'] = df_filtered['Jahr'].astype(str)
    fig_passagiers = px.bar(
        df_filtered,
        x='Jahr',
        y='Flugpassagiere',
        title='Gesamtpassagierzahlen pro Jahr',
        color='Jahr',
        color_discrete_map={str(k): v for k, v in color_map.items()},
        template="plotly_dark"
    )
    return fig_passagiers

# Callback für Polar-Diagramm
@app.callback(
    Output('polar_chart', 'figure'),
    Input('line_checklist', 'value')
)
def update_polar_chart(selected_years):
    filtered_df = df_monate[df_monate['Jahr'].isin(selected_years)].copy()
    filtered_df['Jahr'] = filtered_df['Jahr'].astype(str)
    fig = px.bar_polar(
        filtered_df, 
        r='Flugbewegungen', 
        theta='Monat', 
        color='Jahr', 
        color_discrete_map={str(k): v for k, v in color_map.items()},
        template="plotly_dark"
    )
    fig.update_layout(
       title={"text": "Flugbewegungen pro Monat", "y":0.9},  
       margin=dict(l=60, r=40, t=150, b=25),  
       width=375,  # Feste Breite
       height=450,  # Feste Höhe
       legend=dict(
           y=1.3,  # Verschiebt die Legende nach oben
           yanchor="top",  # Setzt den Anker der Legende nach unten
           x=1,  # Position der Legende auf der x-Achse (rechts)
           xanchor="left")  # Setzt den Anker der Legende nach links
    )
    return fig

# Callback für Scatter-Passagierverkehr
@app.callback(
    Output('scatter_passagierverkehr', 'figure'),
    Input('line_checklist', 'value')
)
def update_scatter_chart(selected_years):
    filtered_df = df[df['Jahr'].isin(selected_years)].copy()
    fig = px.scatter(
        filtered_df,
        x='Flugbewegungen',
        y='Flugpassagiere',
        title="Passagierzahlen - Flugbewegungen",
        color='Jahr',
        template="plotly_dark"
    )
    return fig

if __name__ == "__main__":
    app.run_server(debug=False, host='127.0.0.1', port=9190)